```{contents}
```

## Tool Schema

### What a Tool Schema Is

A **tool schema** is a **machine-readable contract** that describes **how an LLM can call a tool**:

* Tool name
* What it does (description)
* What inputs it accepts (types, required fields, constraints)
* (Optionally) what it returns

> Tool schemas ensure **structured, validated, and safe tool calls**.

---

### Why Tool Schemas Matter

Without schemas:

* LLMs guess arguments
* Inputs are malformed
* Runtime errors occur
* Side effects become unsafe

With schemas:

* Inputs are validated
* Tool calls are deterministic
* Errors are caught early
* Production safety improves

---

### Where Tool Schemas Fit

```
LLM (reasoning)
   ↓
Tool Schema (JSON / Pydantic)
   ↓
Validated Tool Call
   ↓
Tool Execution
   ↓
Result → LLM
```

The LLM **sees only the schema**, not your implementation.

---

### What a Tool Schema Contains

#### Core Fields

1. **name** – unique identifier
2. **description** – guides tool selection
3. **parameters** – input schema (JSON Schema)
4. **required** – mandatory fields
5. **types** – string, number, boolean, enum, object

---

### Example: Tool Schema (Conceptual JSON)

```json
{
  "name": "ticket_lookup",
  "description": "Return number of tickets by source and status",
  "parameters": {
    "type": "object",
    "properties": {
      "source": {
        "type": "string",
        "description": "Ticket system name (jira, servicenow)"
      },
      "status": {
        "type": "string",
        "enum": ["open", "closed"]
      }
    },
    "required": ["source", "status"]
  }
}
```

This schema:

* Prevents extra fields
* Forces required inputs
* Restricts allowed values

---

### Tool Schemas in LangChain (Two Ways)

#### 1) Auto-Inferred Schema (`@tool`)



In [1]:
from langchain_classic.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return total number of tickets for a given source."""
    return 128




LangChain infers:

* `source` → string
* return → int

**Pros:** Simple
**Cons:** Limited validation

---

#### 2) Explicit Schema (StructuredTool + Pydantic) — Recommended



In [3]:
from pydantic import BaseModel, Field
from langchain_classic.tools import StructuredTool

class TicketInput(BaseModel):
    source: str = Field(..., description="Ticket system name")
    status: str = Field(..., description="open or closed")

def ticket_lookup(source: str, status: str) -> int:
    return 128

tool = StructuredTool.from_function(
    name="ticket_lookup",
    description="Return number of tickets by source and status",
    func=ticket_lookup,
    args_schema=TicketInput
)

tool.invoke({"source": "Jira", "status": "open"})

128



**Pros:** Strong validation, production-safe
**Cons:** Slightly more code

---

### How the LLM Uses the Schema

Given the question:

> “How many open Jira tickets are there?”

The LLM emits a **structured call**:

```json
{
  "name": "ticket_lookup",
  "arguments": {
    "source": "jira",
    "status": "open"
  }
}
```

LangChain:

1. Validates against schema
2. Executes tool
3. Returns result to LLM

---

### Validation Failure (Safety Example)

If the LLM tries:

```json
{
  "source": "jira",
  "state": "open"
}
```

Outcome:

* ❌ Schema validation fails
* ❌ Tool not executed
* ❌ No side effects

This is **why schemas are critical**.

---

### Tool Schemas vs Prompt Instructions

| Aspect         | Prompt Instructions | Tool Schemas |
| -------------- | ------------------- | ------------ |
| Enforcement    | Soft                | Hard         |
| Validation     | ❌                   | ✅            |
| Safety         | Low                 | High         |
| Production use | ❌                   | ✅            |

---

### Tool Schemas for Write Actions (Important)

For tools with side effects:

* Create ticket
* Send notification
* Update DB

**Always** use strict schemas with:

* Required fields
* Enums
* Length/range constraints

---

### Best Practices

* Keep schemas minimal
* Use enums for constrained values
* Write clear descriptions
* Separate read vs write schemas
* Log validation errors
* Version schemas when changing inputs

---

### Common Mistakes

* Overloading one tool with many responsibilities
* Weak descriptions (tool misselection)
* Using inferred schemas for write tools
* Allowing free-form strings where enums fit

---

### Interview-Ready Summary

> “Tool schemas define the structured contract between an LLM and a tool, specifying valid inputs and constraints. They enable deterministic, validated, and production-safe tool calling.”

---

### Rule of Thumb

* **Prototype → inferred schemas**
* **Production → explicit schemas (Pydantic)**
* **Write operations → strict schemas only**
* **If safety matters → schema first**
